# 🌋 [PASS Summit](https://pass.org/summit)

## 🐋 Databases

In [4]:
if db_id('FleitasArts') is null create database FleitasArts;
go
if db_id('tpcxbb_1gb') is null
begin
    create database tpcxbb_1gb 
    on  (filename = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\tpcxbb_1gb.mdf'),
        (filename = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\tpcxbb_1gb.ldf')   
    for attach;
end
go 

## 🐋 Config & Grant Access (Restart SQLSvc)

In [5]:
exec sp_configure 'external scripts enabled', 1;
reconfigure with override;
go 
declare @sql nvarchar(max);
select @sql = N'if not exists (select 1 from syslogins where name ='''+ @@servername +'\SQLRUserGroup'') 
begin create login ['+@@servername+'\SQLRUserGroup] from windows end';
exec sp_executesql @sql;
go 

## 🚧 Install MLM (ps as admin)

In [2]:
!powershell "cd C:\fleitasarts\seattle2019\2SentimentPrediction\PS-CMD\
./Install-MLModels.ps1 MSSQLSERVER"

## 🚧 Add To SQL Server Pre-Trained MLM (cmd as admin)

In [0]:
%%cmd
cd "C:\Program Files\Microsoft SQL Server\150\Setup Bootstrap\SQL2019\x64\"
RSetup.exe /install /component MLM /version 9.4.7.0 /language 1033 /destdir "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\Lib\site-packages\microsoftml\mxLibs"

## 🐋 Drop Assemblies (if necessary)

In [2]:
select compatibility_level, name from sys.databases
if exists (select distinct 1 from sys.databases where compatibility_level=150)
begin 
	print N'proceed 😀!' --if 150, proceed!
end 
else begin
	print N'rollup your compat, may need to update stats, if comming from < 2016.' 
end

use FleitasArts
-- use tpcxbb_1gb
go
drop procedure if exists sp_rxPredict;

drop assembly if exists [Microsoft.MachineLearning.RServerScoring.Sql]
drop assembly if exists [System.Windows.Forms.DataVisualization]
drop assembly if exists [System.Windows.Forms]
drop assembly if exists Accessibility

drop assembly if exists [Microsoft.CSharp]
drop assembly if exists [Microsoft.RServer.NativeScorer]
drop assembly if exists [Microsoft.RServer.ScoringLibrary.SqlServer]
drop assembly if exists [Microsoft.RServer.ScoringLibrary]

--drop assembly if exists [Microsoft.SqlServer.Types] -- not permitted for system assemblies, only user ones.
drop assembly if exists [System.Runtime.Serialization]
drop assembly if exists [SMDiagnostics]
drop assembly if exists [system.drawing]
drop assembly if exists [System.Dynamic]

drop assembly if exists [System.IO.Compression.FileSystem]
drop assembly if exists [System.IO.Compression]
drop assembly if exists [System.Runtime.Serialization.Formatters.soap]
drop assembly if exists [System.ServiceModel.Internals]

--wohoo!!!
--done!!

## 🚧 Enable Realtime Predictions (cmd as admin)

In [0]:
%%cmd 
cd "C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\R_SERVICES\library\RevoScaleR\rxLibs\x64\"
RegisterRExt.exe /installRts
RegisterRExt.exe /installRts /database:tpcxbb_1gb
RegisterRExt.exe /installRts /database:FleitasArts

In [5]:
exec sp_configure 'show advanced options', 1;  
reconfigure;
exec sp_configure 'clr enabled', 1;  
reconfigure with override;
go
alter database tpcxbb_1gb set trustworthy on; 
exec sp_changedbowner @loginame = sa, @map = false;
alter database FleitasArts set trustworthy on;
exec sp_changedbowner @loginame = sa, @map = false;

## 🐋 Proc Uses Pre-Trained Models

In [6]:
use FleitasArts;
go
create or alter proc GetSentiment (
    @text nvarchar(max)
)
as 
    declare @script nvarchar(max);
    set @script = N'import pandas as p
from microsoftml import rx_featurize, get_sentiment
analyze_this = text
text_to_analyze = p.DataFrame(data=dict(Text=[analyze_this]))
sentiment_scores = rx_featurize(data=text_to_analyze,ml_transforms=[get_sentiment(cols=dict(scores="Text"))])
sentiment_scores["Sentiment"] = sentiment_scores.scores.apply(lambda score: "Positive" if score > 0.6 else "Negative")'
    exec sp_execute_external_script @language = N'Python',
                                    @script = @script,
                                    @output_data_1_name = N'sentiment_scores',
                                    @params = N'@text nvarchar(max)',
                                    @text = @text
    with result sets (("Text" nvarchar(max), "Score" float, "Sentiment" nvarchar(30)));
go 
exec dbo.GetSentiment N'These are not a normal stress reliever. First of all, they got sticky, hairy and dirty on the first day I received them. Second, they arrived with tiny wrinkles in their bodies and they were cold. Third, their paint started coming off. Fourth when they finally warmed up they started to stick together. Last, I thought they would be foam but, they are a sticky rubber. If these were not rubber, this review would not be so bad.';
exec dbo.GetSentiment N'These are the cutest things ever!! Super fun to play with and the best part is that it lasts for a really long time. So far these have been thrown all over the place with so many of my friends asking to borrow them because they are so fun to play with. Super soft and squishy just the perfect toy for all ages.'
exec dbo.GetSentiment N'I really did not like the taste of it' 
exec dbo.GetSentiment N'It was surprisingly quite good!'
exec dbo.GetSentiment N'I will never ever ever go to that place again!!' 
exec dbo.GetSentiment N'Destiny is a gift. Some go their entire lives, living existence as a quiet desperation. Never learning the truth that what feels as though a burden pushing down upon our shoulders, is actually, a sense of purpose that lifts us to greater heights. Never forget that fear is but the precursor to valor, that to strive and triumph in the face of fear, is what it means to be a hero. Don''t think, Master Jim. Become!';

## 🐋 In-Database ML (memory + cpu)

In [7]:
use [tpcxbb_1gb]
go
drop table if exists models
go
create table models (
	 language		varchar(30) not null
	,model_name		varchar(30) not null
	,model			varbinary(max) 
	,create_time	datetime default(getdate())
	,created_by		nvarchar(500) default(suser_sname())
 primary key clustered (language, model_name)
)
go

drop view if exists product_reviews_training_data;
go
create or alter view product_reviews_training_data
as
	select	top(select cast(count(*)*.9 as int) from product_reviews)
			cast(pr_review_content as nvarchar(4000)) as pr_review_content
			,case
				when pr_review_rating < 3 then 1 
				when pr_review_rating = 3 then 2 else 3 
			end as tag 
	from	product_reviews;
go

drop view if exists product_reviews_test_data;
go
create or alter view product_reviews_test_data
as 
	select	top(select cast(count(*)*.1 as int) from product_reviews)
			cast(pr_review_content as nvarchar(4000)) as pr_review_content
			,case
				when pr_review_rating < 3 then 1 
				when pr_review_rating = 3 then 2 else 3 
			end as tag 
	from	product_reviews;
go

-- 1 = Negative, 2 = Neutral, 3 = Positive
create or alter proc create_text_classification_model
as
	declare @model varbinary(max), @train_script nvarchar(max);
	
	--The Python script we want to execute
	set @train_script = N'
##Import necessary packages
from microsoftml import rx_logistic_regression, featurize_text, n_gram
import pickle

training_data["tag"] = training_data["tag"].astype("category")

#ngramLength=2: include not only "Word1", "Word2", but also "Word1 Word2"
#weighting="TfIdf": Term frequency & inverse document frequency
model = rx_logistic_regression(formula = "tag ~ features", data = training_data, method = "multiClass", ml_transforms=[
                        featurize_text(language="English",
                                     cols=dict(features="pr_review_content"),
                                      word_feature_extractor=n_gram(2, weighting="TfIdf"))]) 
modelbin = pickle.dumps(model)';

	execute sp_execute_external_script @language = N'Python'
		,@script = @train_script
		,@input_data_1 = N'select * from product_reviews_training_data'
		,@input_data_1_name = N'training_data'
		,@params  = N'@modelbin varbinary(max) OUTPUT' 
		,@modelbin = @model output;
 
	--Save model to DB Table
	delete from models where model_name = 'rx_logistic_regression' and language = 'Python';
	insert into models (language, model_name, model) values('Python', 'rx_logistic_regression', @model);
go
exec create_text_classification_model;
select * from dbo.models;
go

create or alter proc predict_review_sentiment 
as
	declare @model_bin varbinary(max), @prediction_script nvarchar(max);
	select @model_bin = model from dbo.models where model_name = 'rx_logistic_regression' and language = 'Python';
 
	set @prediction_script = N'
from microsoftml import rx_predict
from revoscalepy import rx_data_step 
import pickle

model = pickle.loads(model_bin)
predictions = rx_predict(model = model, data = test_data, extra_vars_to_write = ["pr_review_content"], overwrite = True)

result = rx_data_step(predictions)
## print(result)';
 
	exec sp_execute_external_script @language = N'Python'
		,@script = @prediction_script
		,@input_data_1 = N'select * from product_reviews_test_data'
		,@input_data_1_name = N'test_data'
		,@output_data_1_name = N'result'
		,@params  = N'@model_bin varbinary(max)'
		,@model_bin = @model_bin
	with result sets(("Review" nvarchar(max), "PredictedLabel" float, "Predicted_Score_Negative" float, "Predicted_Score_Neutral" float, "Predicted_Score_Positive" float)); 
go
-- The predicted score of Negative means the statement is (x percent Negative), and so on for the other sentiment categories. 
-- Ie. since there all tag 3 positive, they will have very low negative scores, low neutral scores and very high positive scores. 
if exists(select 1 from dbo.models where model_name = 'rx_logistic_regression' and language = 'Python' and model is not null)
begin
	exec predict_review_sentiment;
end
go

## 💯 Real-time Scoring

In [1]:
alter table models add default 'Python' for language;
go
create or alter proc CreatePyModelRealtimeScoringOnly 
as
	declare @model varbinary(max), @train_script nvarchar(max);
	delete top(1) from models where model_name = 'RevoMMLRealtimeScoring' and language = 'Py';
	
	set @train_script = N'
from microsoftml import rx_logistic_regression, featurize_text, n_gram
from revoscalepy import rx_serialize_model, RxOdbcData, rx_write_object, RxInSqlServer, rx_set_compute_context, RxLocalSeq

connection_string = "Driver=SQL Server;Server=localhost;Database=tpcxbb_1gb;Trusted_Connection=true;"
dest = RxOdbcData(connection_string, table = "models")
 
training_data["tag"] = training_data["tag"].astype("category")

modelpy = rx_logistic_regression(formula = "tag ~ features",
								 data = training_data, 
								 method = "multiClass", 
								 ml_transforms=[featurize_text(language="English",
															   cols=dict(features="pr_review_content"),
															   word_feature_extractor=n_gram(2, weighting="TfIdf"))],
								 train_threads=1)

modelbin = rx_serialize_model(modelpy, realtime_scoring_only = True)
rx_write_object(dest, key_name="model_name", key="RevoMMLRealtimeScoring", value_name="model", value=modelbin, serialize=False, compress=None, overwrite=False)'; --overwrite=false on 2019, true on 2017.

	exec sp_execute_external_script @language = N'Python'
		,@script = @train_script
		,@input_data_1 = N'select * from product_reviews_training_data'
		,@input_data_1_name = N'training_data'
go
exec  CreatePyModelRealtimeScoringOnly; --00:01:14.560 desktop, 00:02:40.351 laptop.
select *, datalength(model) as Datalen from dbo.models; --(6MB w/rx_write_object vs 55MB w/pickle.dump)
go
declare @model_bin varbinary(max)=null
select	@model_bin = model from models where model_name = 'RevoMMLRealtimeScoring';
if @model_bin is not null 
begin
    exec sp_rxPredict @model = @model_bin, @inputData = N'select pr_review_content, cast(tag as varchar(1)) as tag from product_reviews_test_data' 
end;
go --8,999 rows: sp_rxPredict 3-9sec vs python microsoftml rx_predict 11-25sec.